# Introdução

O projeto, da unidade curricular de Inteligência Artificial da Licenciatura em Engenharia de Sistemas Informáticos (2024-25), tem como objetivo implementar abordagens e métodos distintos de Aprendizagem Automática (ML) para resolver um problema específico utilizando um conjunto de dados públicos.

O trabalho foi realizado pelos seguintes membros:

- João Ponte - 17694
- João Carvalho - 12747

Para a realização deste projeto foi selecionado o seguinte dataset público:
- https://www.kaggle.com/code/ryanholbrook/clustering-with-k-means/tutorial

Link para o repositório do projeto no Github:
- https://github.com/a12747/IA24_G11_2

# Desenvolvimento

## Instalação de livrarias

In [1]:
!pip install matplotlib
!pip install seaborn
!pip install xgboost
!pip install mlxtend

## Importação de livrarias

In [2]:
import pandas as pd
import numpy as np

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn para ML
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Modelos de Classificação
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# XGBoost
from xgboost import XGBClassifier

# Para Regras de Associação
from mlxtend.frequent_patterns import apriori, association_rules

## Carregamento de Ficheiro

In [3]:
# Carregar dados
df = pd.read_csv("./datasets/housing.csv")

# Visualizar as 5 primeiras linhas
df.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


# Conclusão

O desenvolvimento deste projeto permitiu aplicar conceitos fundamentais de Inteligência Artificial na criação e implementação de um agente inteligente. 
Durante este processo, foi possível explorar técnicas de planeamento, otimização e resolução de problemas sob restrições, consolidando conhecimentos teóricos e práticos.
Foi programado um agente inteligente capaz de lidar com restrições fortes, como precedência de tarefas e capacidade de recursos, e restrições leves, como a minimização da duração total do calendário (makespan).
Estas restrições foram integradas de maneira a criar um modelo realista e aplicável a cenários complexos.
Ao longo do processo, explorámos diversos tipos de soluções, desde soluções possíveis, que podem violar restrições, até soluções viáveis e otimizadas, que garantem o cumprimento das condições definidas.
O agente foi desenhado para encontrar soluções com pontuações elevadas dentro de um tempo limitado, demonstrando a capacidade de identificar a melhor solução encontrada e, quando possível, soluções ótimas.
